Zygote exports 5 interfaces. All can be found in [src/compiler/interface.jl](https://github.com/FluxML/Zygote.jl/blob/master/src/compiler/interface.jl)

1. [Params](https://github.com/FluxML/Zygote.jl/blob/7c1ad4194e/src/compiler/interface.jl#L77)
    ```julia
    struct Params
      params::IdSet{Any}
      Params(xs) = new(IdSet(xs))
    end
    ```

1. [gradient]()
    ```julia
    function gradient(f, args...)
      y, back = forward(f, args...)
      y isa Real || error("Function output is not scalar")
      return back(Int8(1))
    end
    ```

1. derivative
    ```julia
    derivative(f::F, x) where F = gradient(f, x)[1]
    ```

1. [forward](https://github.com/FluxML/Zygote.jl/blob/master/src/compiler/interface.jl#L36)

    There are two function overloadings of `forward`.
    
    ```julia
    function forward(f, args...)
      y, back = _forward(f, args...)
      y, Δ -> tailmemaybe(back(Δ))
    end

    function forward(f, ps::Params)
      cx = Context()
      y, back = _forward(cx, f)
      y, function (Δ)
        for p in ps
          cache(cx)[p] = nothing
        end
        back(Δ)
        Grads(cx.cache) # TODO make a copy
      end
    end
    ```

1. [@code_grad](https://github.com/FluxML/Zygote.jl/blob/7c1ad4194e1857f213b4955f89315cffe4543d2f/src/compiler/interface.jl#L83)
    Comment by the author.

In [2]:
using Zygote

W, b = rand(2, 3), rand(2)
y(x) = W * x .+ b

ps = Params([W, b])

# Do block and and anonymous function
# https://docs.julialang.org/en/v1/manual/functions/index.html#Do-Block-Syntax-for-Function-Arguments-1
g, _ = gradient(ps) do
  sum(y([1, 2, 3]))
end

g[W], g[b]

([1.0 2.0 3.0; 1.0 2.0 3.0], [1.0, 1.0])

In [4]:
# How to use gradient

g = gradient((x, y) -> sum(x .* y), [1, 2, 3], [4, 5, 6])

g[1] ## this is the gradient of the two inputs
@code_llvm g[2]

# this is the gradient function which has 1 input: the recieved gradient
g[2](2)


; Function getindex
; Location: tuple.jl:24
define nonnull %jl_value_t addrspace(10)* @julia_getindex_37280(%jl_value_t addrspace(10)* nonnull align 8 dereferenceable(16), i64) {
top:
  %2 = add i64 %1, -1
  %3 = icmp ult i64 %2, 2
  br i1 %3, label %pass, label %fail

fail:                                             ; preds = %top
  %4 = addrspacecast %jl_value_t addrspace(10)* %0 to %jl_value_t addrspace(12)*
  call void @jl_bounds_error_int(%jl_value_t addrspace(12)* %4, i64 %1)
  unreachable

pass:                                             ; preds = %top
  %5 = bitcast %jl_value_t addrspace(10)* %0 to %jl_value_t addrspace(10)* addrspace(10)*
  %6 = addrspacecast %jl_value_t addrspace(10)* addrspace(10)* %5 to %jl_value_t addrspace(10)* addrspace(11)*
  %7 = getelementptr inbounds %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)* addrspace(11)* %6, i64 %2
  %8 = load %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)* addrspace(11)* %7, align 8
  ret %jl_value_t addrspa

([8, 10, 12], [2, 4, 6])

```julia
@generated function _forward(Δ::Tuple, f, args...)
  T = Tuple{f,args...}
  meta = IRTools.meta(T)
  forw = record(IR(meta))
  pushfirst!(forw.args, Any, Any)
  argnames!(meta, Symbol("#self#"), :Δ, :f, :args)
  forw = varargs!(meta, forw, 3)
  forw = slots!(forw)
  return IRTools.update!(meta, forw)
end
```